In [79]:
a = open("Articles.txt",'r')
b = a.read()
articles=[]
text=""
title=""
titles=[]
for i in b.split("-------------------"):
    if i =="":
        continue
    article = i.split('\n\n\n\n\n')
    text = article[1].strip('\n').strip(' ')
    if text.count('\n') <5:
        continue
    title = article[0].strip('\n').strip(' ')
    if title in titles:
        continue
    titles.append(title)
    articles.append([title,text])
a.close()

In [80]:
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
ps = SnowballStemmer('english')
wordRepArticles =[]
idf={}
articleslen=[]
for i in articles:
    title,text = i[0],i[1]
    text = text.replace('\\','');text = text.replace('.\n','\n');text = text.replace('\n\n','\n')
    articledist={}
    sentencelis=[]
    count=0
    for i in (text+'\n'+title).split('\n'):
        if i ==title:
            mult =2
        else:
            mult =1
        linedist={}
        v = i.split(' ')
        if len(v)<4:
            continue
        for word in v:
            if word not in stopwords.words('english'):
                word = word.replace('(','');word = word.replace(')','')
                word = word.replace('"','');word = word.replace(',','');word = word.strip(' ')
                stemmed = ps.stem(word)
                if stemmed in articledist.keys():
                    articledist[stemmed]+=mult
                else:
                    articledist[stemmed]=mult
                if stemmed in linedist.keys():
                    linedist[stemmed]+=1
                else:
                    linedist[stemmed]=1
            count+=1
        sentencelis.append(linedist)
    for words in articledist.keys():   
        if words in idf.keys():
            idf[words]+=1
        else:
            idf[words]=1
    wordRepArticles.append([title,articledist,sentencelis])
    articleslen.append(count)

In [81]:
len(wordRepArticles)
# wordRepArticles[0]
len(articles)

434

In [82]:
articles[:5]

[["Senegal gets Africa's first NBA training center",
  'Lagos (CNN) The National Basketball Association (NBA) on Monday commissioned a new training center in Saly, Senegal, the first of its kind in Africa, as part of its drive to attract young teenagers to the league.\n\nThe new facility, which features two indoor basketball courts, a multidimensional activity center, a weight room, conference rooms as well as dormitories and educational facilities, will be the primary training location for NBA Academy Africa future hopefuls from across the continent, the league said in a statement.\n\n"We are thrilled to open this new world-class venue in Saly for NBA Academy Africa prospects," said Amadou Gallo Fall, vice president and managing director of NBA Africa.\n\n"This facility will provide aspiring players from across the continent with state-of-the-art courts and training equipment, under the tutelage of our NBA coaches and our world-class academic staff," he said.\n\nThe NBA Academy Africa

In [83]:
import math
n = len(articles)
avgArticleLen = sum(articleslen)*1.0/n
match = [[0 for i in range(n)] for i in range(n)]
for i in range(len(articles)):
    mainarticle = wordRepArticles[i]
    mainlength = articleslen[i]
    for j in range(i+1,len(articles)):
        comparearticle = wordRepArticles[j]
        comparelength = articleslen[j]
        commonwords = list(set(mainarticle[1].keys()) & set(comparearticle[1].keys())) 
        value = 0
        for word in commonwords:
            mainvalue = (1+math.log(mainarticle[1][word]))*1.0/(math.log(1+mainlength/avgArticleLen))
            compvalue = (1+math.log(comparearticle[1][word]))*1.0/(math.log(1+comparelength/avgArticleLen))
            value += mainvalue*compvalue* math.log(1+n/idf[word])
        match[i][j] = value
        match[j][i] = value

In [84]:
matches = 3
for i in range(n):
    vals = [[match[i][j],j] for j in range(n)]
    toparticle = sorted(vals,reverse=True)[:matches]
    print("Main title: "+ articles[i][0])
    for j in range(matches):
        print(str(j)+"/" + str(matches)+ ": " + articles[toparticle[j][1]][0])

Main title: Senegal gets Africa's first NBA training center
0/3: Tijuana Shuts Down Migrant Caravan Shelter
1/3: How South Africa built one of the world's most advanced telescopes
2/3: On the hunt for India’s ‘stolen’ heritage
Main title: Police arrest 6 after a teenage boy died in an adulthood initiation ritual in Kenya
0/3: Sister of attacked teenage Syrian refugee also targeted by bullies in UK school
1/3: UK student to be charged after Syrian refugee attacked
2/3: Woman killed 'leaning out of train window' between Bath and Bristol
Main title: At least 35 people are dead and dozens are missing after a boat accident in Uganda
0/3: Shock Video: Migrants Land Boat On California Beach 100 Miles From Border
1/3: Woman killed on train may have been leaning out of window, say police
2/3: Woman killed 'leaning out of train window' between Bath and Bristol
Main title: US Embassy in DRC warns of "possible terrorist threat" against US facilities
0/3: Latest Ebola outbreak is now the worst, aft

0/3: Dem Congresswoman Admits Helping Caravan Members Enter US After They Were Denied Entry
1/3: Trump Closes Major Trade Deal With Canada, Mexico
2/3: Egyptian actress could face 5 years in prison for wearing revealing dress
Main title: Immigration: The conversation we need to be having around America’s kitchen tables -- but aren’t
0/3: “Now or Never”: Why the Migrant Caravan Plans Second Border Strike Saturday
1/3: Breaking: Migrant Caravan Set To Storm US Border Saturday Due To Mexico Eviction
2/3: Dem Congresswoman Admits Helping Caravan Members Enter US After They Were Denied Entry
Main title: Rabbi Tuly Weisz: Hanukkah is not just a Festival of Lights, but a Festival of Miracles
0/3: President Trump and First Lady Melania Wish Jews a Happy Hanukkah
1/3: Netanyahu: We Will Always Remember Bush’s Commitment to Israel
2/3: At Yanar Dag, see why Azerbaijan is the 'Land of Fire'
Main title: Wondering if you'll ever find 'the one'? Here's the best thing you can do
0/3: If I Die, I Want

In [70]:
math.log2(1.1)

0.13750352374993502

In [71]:
print(avgArticleLen)

552.3788343558282
